In [136]:
%load_ext autoreload
%autoreload 2

from src.wmf import WeightedMatrixFactorization
import numpy as np

feedback_matrix = np.array([[3, 4, np.nan, 1, np.nan, np.nan, 5, np.nan, np.nan, np.nan],
                            [1, 3, np.nan, np.nan, np.nan, 3, np.nan, np.nan, 5, 5],
                            [1, np.nan, 1, 4, 4, np.nan, np.nan, np.nan, 3, 4]])

wmf = WeightedMatrixFactorization(
    feedback_matrix, 
    n_latents=100,
    n_iter=100,
    lambda_reg=0.05
    )

hist = wmf.fit(method='WALS', verbose=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loss: 0.151, iteration: 1/100
Loss: 0.130, iteration: 2/100
Loss: 0.119, iteration: 3/100
Loss: 0.111, iteration: 4/100
Loss: 0.106, iteration: 5/100
Loss: 0.102, iteration: 6/100
Loss: 0.098, iteration: 7/100
Loss: 0.095, iteration: 8/100
Loss: 0.093, iteration: 9/100
Loss: 0.091, iteration: 10/100
Loss: 0.089, iteration: 11/100
Loss: 0.087, iteration: 12/100
Loss: 0.085, iteration: 13/100
Loss: 0.084, iteration: 14/100
Loss: 0.082, iteration: 15/100
Loss: 0.081, iteration: 16/100
Loss: 0.080, iteration: 17/100
Loss: 0.079, iteration: 18/100
Loss: 0.078, iteration: 19/100
Loss: 0.077, iteration: 20/100
Loss: 0.076, iteration: 21/100
Loss: 0.075, iteration: 22/100
Loss: 0.075, iteration: 23/100
Loss: 0.074, iteration: 24/100
Loss: 0.074, iteration: 25/100
Loss: 0.073, iteration: 26/100
Loss: 0.072, iteration: 27/100
Loss: 0.072, iteration: 28/100
Loss: 0.072, iteration: 29/100
Loss: 0.071, iteration

In [132]:
print(hist.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])


In [133]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')
# plot the history of the loss function

sns.lineplot(x=hist.keys(), y=hist['loss'])
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()

KeyError: 'loss'